In [ ]:
# Black
import re
import nltk
import csv
import pandas as pd
import math
import sys

upVerbs = ["went up","increased","rose"]
downVerbs = ["went down","decreased","fell"]

df = pd.read_csv("input.csv",header=None,dtype=str)
df = df.fillna('missing')

#find error
cntErr = 0
errTex = []

if df.at[0,0] == 'missing':
    errTex.append('Row 0, column 0 : Subject is missing')
    cntErr += 1
    
for i in range(1,len(df)):
    for j in [0,1]:
        p = 'Row ' + str(i) + ', column ' + str(j) + ' : '
        if df.at[i,j] == 'missing':
            errTex.append(p + 'Value is missing')
            cntErr += 1
        elif j == 0 and df.at[i,j].isdigit():
            errTex.append(p + 'Please enter a time period')
            cntErr += 1
        elif j == 1 and not df.at[i,j].isdigit():
            errTex.append(p + 'Please enter a number')
            cntErr += 1

if cntErr > 0:
    print(str(cntErr) + ' error'+ ("s" if cntErr > 1 else "") + ' found')
    for i in errTex:
        print(i)
    sys.exit()

#extract nouns
tokens = nltk.word_tokenize(df.at[0,0])
tagged = nltk.pos_tag(tokens)
nouns = [df.at[0,0],"They"]

for i in tagged:
    if i[1] in ['NN','NNS','NNP','NNPS']:
        nouns.append("The " + i[0])
        if i[1] in ['NN','NNP']:
            nouns.append("It")
        else:
            nouns.append("They")

#extract values
P = [int(i) for i in df.iloc[1:,1]]
M = [i for i in df.iloc[1:,0]]
    
#extract max value and min value
maxValue = P[0]
minValue = P[0]
for i in P:    
    maxValue = max(maxValue,i)
    minValue = min(minValue,i)
    
#remove elements with similar tendencies
newP = [P[0]]
newM = [M[0]]
for i in range(1,len(P)-1):
    sub1 = P[i] - P[i-1]
    sub2 = P[i+1] - P[i]
    if sub1 * sub2 > 0:
        continue
    if sub1 == sub2 == 0:
        continue
    newP.append(P[i])
    newM.append(M[i])
    
newP.append(P[-1])
newM.append(M[-1])
P = newP
M = newM

#generate subject
cntEven = 0
flagMax = 0
flagMin = 0

for i in range(0,len(P)-1):    
    
    cntNoun = i % len(nouns)
    cntVerb = i % len(upVerbs)
   
    P1 = P[i]
    P2 = P[i+1]
    M1 = M[i]
    M2 = M[i+1]
     
    #when the value does not change
    if P1 == P2:
        
        subject = nouns[cntNoun]        
        
        if cntEven % 3 == 0:
            subject += " remained stable at "
        elif cntEven % 3 == 1:
            subject += " remained steady at "
        else:
            subject += " evened out "
            
        subject += str(P1) + " from " + M1 + " to " + M2 + "."
        print(subject)        
        continue
    
    #rate of change
    rate = 1
    if P1 > 0:
        rate = float(abs(P1 - P2)) / float(P1)
    
    if rate <= 0.05:
        adj = "slight"
    elif rate >= 1:
        adj = "significant"
    else:
        adj = "substantial"
        
    #passive judgment and verb addition
    if i % 4 == 3:
        subject = "There was a " + adj        
        if P1 < P2:
            subject += " increase"
        else:
            subject += " decrease"  
    else:
        subject = nouns[cntNoun] + " "      
        if P1 < P2:
            subject += upVerbs[cntVerb]
        else:
            subject += downVerbs[cntVerb]
        subject += " " + adj + "ly"
    
    #max or min
    S1 = str(P1)
    S2 = str(P2)
    
    if P1 == maxValue:
        if flagMax == 0:
            S1 = "a high of " + S1
            flagMax = 1
        else:
            S1 = "its peak"
    
    if P1 == minValue:
        if flagMin == 0:
            S1 = "a low of " + S1
            flagMin = 1
        else:
            S1 = "its low"
            
    if P2 == maxValue:
        if flagMax == 0:
            S2 = "a high of " + S2
            flagMax = 1
        else:
            S2 = "its peak"
            
    if P2 == minValue:
        if flagMin == 0:
            S2 = "a low of " + S2
            flagMin = 1
        else:
            S2 = "its low"
    
    #time
    if i % 5 == 0:
        subject += " from " + S1 + " in " + M1 + " to " + S2 + " in " + M2 + "."
    elif i % 5 == 1:
        subject += " from " + S1 + " in " + M1 +" by " + str(abs(P1-P2)) + " to " + S2 + " in " + M2 + "."
    elif i % 5 == 2:
        subject += " in " + M1 + " by " + str(abs(P1-P2)) + " to " + S2 + " in " + M2 + "."
    elif i % 5 == 3:
        subject += " from " + S1 + " in " + M1 +" by " + str(abs(P1-P2))
        if S2[0] == 'a':
            subject += " to " + S2
        subject += " in " + M2 + "."
    else:
        subject = "From " + M1 + " to " + M2 + ", " + subject.lower() + " to " + S2
                                                                                
    print(subject)